this code was to use my kaggle api key to download the dataset in here but i dont think it would work for you guys so ignore it and just download the dataset manually lol

In [1]:
# copy kaggle.json to /root/.kaggle/ folder so that kaggle cli can access it.
# !mkdir /.kaggle
# !mv kaggle.json /.kaggle
# !mv /.kaggle /root/
# !chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/.kaggle’: File exists
mv: cannot stat 'kaggle.json': No such file or directory
mv: cannot move '/.kaggle' to '/root/.kaggle': Directory not empty


In [2]:
# !kaggle datasets download -d mohamedbakhet/amazon-books-reviews

amazon-books-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
# !unzip amazon-books-reviews.zip

In [1]:
import pandas as pd

books_data = pd.read_csv('books_data.csv')
books_ratings = pd.read_csv('Books_rating.csv')


In [5]:
books_data.shape

(212404, 10)

In [6]:
books_ratings.shape
books_ratings.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


# this turns the review helpfulness into a numerical value
# maybe useful to decide which reviews were good?

In [ ]:
books_ratings['review/helpfulness'].value_counts()

ratings = []
for rating in books_ratings['review/helpfulness']:
  numerator, denominator = rating.split('/')

  numerator = int(numerator)
  denominator = int(denominator)

  if(numerator == 0 or denominator == 0):
    continue

  ratings.append(round(numerator / denominator, 2))

ratings

merge and clean the two datasets

In [2]:
merged_books = pd.merge(books_data, books_ratings, on='Title')
merged_books.shape

(3000000, 19)

In [3]:
merged_books_sample = merged_books.sample(10_000)

In [4]:
merged_books_sample_reduced = merged_books_sample[['Title', 'description', 'categories', 'User_id', 'review/score', 'review/summary', 'review/text']]
merged_books_sample_reduced.head()

,Title,description,categories,User_id,review/score,review/summary,review/text
2303785,Mother Of The Groom: Everything you Need to Kn...,It’s not just about the rehearsal dinner anymo...,['Reference'],A2IJ2XO0NS3FLR,5.0,Great Gift for your Groom's Mom!,I bought this as a gift for my fiance's mom. I...
1423289,Of Mice and Men,NaN,NaN,A1QWPZQW5FRVE0,5.0,Of Mice and Men GT Review,Of Mice and Men starts out as a quite confusin...
1609014,The Fulfillment of All Desire,Winner: Honorable Mention from the Catholic Pr...,['Spiritual life'],A32UUBSGHW403P,5.0,Seven Spiritual Directors - The Doctors are In!,I am still reading this book - but I read a pa...
205832,Breakfast of Champions,Discover Vonnegurt’s funny absurdist novel abo...,['Fiction'],NaN,5.0,"Great book, true vonnegut form",I am an avid reader of kurt vonnegut's work an...
1276653,I Know Why the Caged Bird Sings,"Here is a book as joyous and painful, as myste...",['Biography & Autobiography'],A1ASUSV2FNDALE,5.0,Very Well Written,This book is very well written. The words and ...


In [5]:
df_cleaned = merged_books_sample_reduced.drop_duplicates()


In [12]:
df_cleaned.isna().sum()

Title                10
description       21413
categories        18449
User_id           18672
review/score          0
review/summary        2
review/text           0
dtype: int64

In [6]:
df_cleaned.dropna(inplace = True)
df_cleaned.shape

<ipython-input-6-2f76804402ef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.dropna(inplace = True)


(6027, 7)

In [14]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59783 entries, 587385 to 489956
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Title           59783 non-null  object 
 1   description     59783 non-null  object 
 2   categories      59783 non-null  object 
 3   User_id         59783 non-null  object 
 4   review/score    59783 non-null  float64
 5   review/summary  59783 non-null  object 
 6   review/text     59783 non-null  object 
dtypes: float64(1), object(6)
memory usage: 3.6+ MB


In [7]:
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
# import string
# from string import punctuation
# from itertools import chain

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, f1_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_selection import SelectPercentile, chi2, f_regression, f_classif

# from sklearn import svm
from sklearn.utils import shuffle

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


this was a sentiment analysis library i found, maybe it can be useful later? who knows yes

In [16]:
# !pip install  vaderSentiment

In [17]:
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# # Initialize the sentiment analyzer
# analyzer = SentimentIntensityAnalyzer()

# # Sample text
# text = "VADER is a serviceable tool for sentiment analysis."

# # Analyze sentiment
# sentiment_scores = analyzer.polarity_scores(text)

# # Print sentiment scores
# print(sentiment_scores)

In [ ]:
df_cleaned.head()

this is just another dataframe i made to mess around with in the following code

In [61]:
df = df_cleaned[['Title', 'review/summary', 'User_id', 'review/text']]
df.head()

,Title,review/summary,User_id,review/text
2303785,Mother Of The Groom: Everything you Need to Kn...,Great Gift for your Groom's Mom!,A2IJ2XO0NS3FLR,I bought this as a gift for my fiance's mom. I...
1609014,The Fulfillment of All Desire,Seven Spiritual Directors - The Doctors are In!,A32UUBSGHW403P,I am still reading this book - but I read a pa...
1276653,I Know Why the Caged Bird Sings,Very Well Written,A1ASUSV2FNDALE,This book is very well written. The words and ...
1998994,The DHEA Breakthrough,DHEA - amazing information,A32EHNK1TVFA8L,Amazing book - there's so much information abo...
1436743,We Were the Mulvaneys,it was ok,A3IV0PEZJMP2PS,i wouldn't read it again. or for that matter r...


this cleans the text passed in to make things easier

In [57]:
def clean_text(text):
  # replacements = {'-': ' ', '!': ' ', ',': ' '}
  # df['review/summary'].str.lower().replace(['-', '!'], ' ')

  # df['review/summary'] = df['review/summary'].replace(replacements, regex=True)
  # df['review/summary'] = df['review/summary'].str.lower()
  # df = df.reset_index()
  # df.head(15)
  text = str(text)
  text = text.replace('-', ' ')
  text = text.replace('!', ' ')
  text = text.replace(',', ' ')
  text = text.replace('?', ' ')
  text = text.replace(':', ' ')
  text = text.replace("'", ' ')
  text = text.lower()
  return(text)

clean the titles, reviews/text

In [62]:
df['Title'] = df['Title'].apply(clean_text)
df['Title'].head()

<ipython-input-62-d28b221e7dd7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Title'] = df['Title'].apply(clean_text)


2303785    mother of the groom  everything you need to kn...
1609014                        the fulfillment of all desire
1276653                      i know why the caged bird sings
1998994                                the dhea breakthrough
1436743                                we were the mulvaneys
Name: Title, dtype: object

In [63]:
df['review/text'] = df['review/text'].apply(clean_text)

<ipython-input-63-d11d9d47bc4a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review/text'] = df['review/text'].apply(clean_text)


reset the index because the whole thing breaks otherwise lol

In [65]:
df = df.reset_index()

In [67]:
df.head()

,index,Title,review/summary,User_id,review/text
0,2303785,mother of the groom everything you need to kn...,Great Gift for your Groom's Mom!,A2IJ2XO0NS3FLR,i bought this as a gift for my fiance s mom. i...
1,1609014,the fulfillment of all desire,Seven Spiritual Directors - The Doctors are In!,A32UUBSGHW403P,i am still reading this book but i read a pa...
2,1276653,i know why the caged bird sings,Very Well Written,A1ASUSV2FNDALE,this book is very well written. the words and ...
3,1998994,the dhea breakthrough,DHEA - amazing information,A32EHNK1TVFA8L,amazing book there s so much information abo...
4,1436743,we were the mulvaneys,it was ok,A3IV0PEZJMP2PS,i wouldn t read it again. or for that matter r...


this is the main part

TO TRY OUT DIFFERENT BOOKS YOURSELF, JUST CHANGE THE 'title' VARIABLE AT THE BOTTOM OF THE CELL TO WHATEVER TITLE YOU SEE ***IN THE DATASET***

i had to limit how much items are in the dataset to just a sample of 10,000 because colab was literally running out of ram and crashing, so you can only enter titles that are in the actual sampled dataset which is random every time. call the dataset and pick a title you see to use it

roughly how it works:
- vectorize the feature you want to find similarities in, in this case i used the review/text
- then use linear_kernel function to get the cosine similarity score
- the function takes in the book title you want to find recomendations for and the cosine score, then finds what index the book title you are searching for is, then gets all the cosine similarity scores for that book, sorts in descending order and gives you the top 10

things i was thinking to MAYBE improve this because i think this has some promise to it but it is clearly flawed:

1) find the similarity scores for multiple features then cross reference them to find the books that have the highest scores across the different featuers. Like if you try to find it for the reviews AND description, and then pull the top 10 books that are the highest in both? maybe that can give more accurate books

2) somehow encorporate genres into it so you can limit the range of similar books?

3) some kind of sentimental analyis to also help narrow the range of similar books? maybe only similar books that have a same sentimental score?


who knows, i just think this is promising so far and can maybe lead to something


In [80]:
from sklearn.metrics.pairwise import linear_kernel


# df = df_cleaned[['Title', 'review/summary', 'User_id']]

# test_df = df_cleaned[['Title', 'review/summary', 'User_id']]

# Preprocessing and vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['review/text'])

# Compute cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Function to recommend similar books
def recommend_books(title, cosine_sim=cosine_sim):

    match_row = df[df['Title'] == title]

    if match_row.empty:
      return('No match')

    idx = df[df['Title'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Exclude the first item (itself) and take top 10
    book_indices = [i[0] for i in sim_scores]
    return df['Title'].iloc[book_indices]

# Example usage
title = "the way of the cross"
title = clean_text(title)
recommendations = recommend_books(title)
print("Recommendations for", title, ":")
print(recommendations)

Recommendations for the way of the cross :
3239                     cross stitch
4696                     cross stitch
2186    the cross and the switchblade
749     the art of cross examination 
4460     the art of cross examination
4563    the art of cross examination 
1352                     the way west
147                   the martian way
4205                  the martian way
1544            the way of all flesh 
Name: Title, dtype: object


In [30]:
df[df['Title'] == 'The Fulfillment of All Desire'].index[0]

1

In [79]:
cosine_sim[1]

array([0.02564313, 1.        , 0.01326567, ..., 0.01791944, 0.01412891,
       0.00513833])